In [10]:
import tkinter as tk
from tkinter import messagebox

# Datos de inicio de sesión
usuario_correcto = "admin123"
contrasena_correcta = "contrasena123"

def verificar():
    if usuario.get() == usuario_correcto and contrasena.get() == contrasena_correcta:
        messagebox.showinfo("", "Inicio de sesión exitoso")
        ventana_login.destroy()
    else:
        messagebox.showinfo("", "Usuario o contraseña incorrectos")

ventana_login = tk.Tk()
ventana_login.title("Inicio de sesión")

usuario = tk.StringVar()
contrasena = tk.StringVar()

tk.Label(ventana_login, text="Usuario").pack()
tk.Entry(ventana_login, textvariable=usuario).pack()

tk.Label(ventana_login, text="Contraseña").pack()
tk.Entry(ventana_login, textvariable=contrasena, show='*').pack()

tk.Button(ventana_login, text="Iniciar sesión", command=verificar).pack()

ventana_login.mainloop()


In [12]:
import tkinter as tk
from tkinter import messagebox
import json

class Modelo:
    def __init__(self):
        self.usuarios = {"admin123": "contrasena123"}
        self.cargar_pacientes()

    def verificar_usuario(self, usuario, contrasena):
        return usuario in self.usuarios and contrasena == self.usuarios[usuario]

    def agregar_paciente(self, id, nombre, apellido, edad):
        if id in self.pacientes:
            return False
        self.pacientes[id] = {"nombre": nombre, "apellido": apellido, "edad": edad}
        self.guardar_pacientes()
        return True

    def eliminar_paciente(self, id):
        if id in self.pacientes:
            del self.pacientes[id]
            self.guardar_pacientes()
            return True
        return False

    def buscar_pacientes(self, nombre):
        return {id: datos for id, datos in self.pacientes.items() if datos["nombre"].lower().startswith(nombre.lower())}

    def cargar_pacientes(self):
        try:
            with open('pacientes.json', 'r') as f:
                self.pacientes = json.load(f)
        except FileNotFoundError:
            self.pacientes = {}

    def guardar_pacientes(self):
        with open('pacientes.json', 'w') as f:
            json.dump(self.pacientes, f)

class Vista:
    def __init__(self, controlador):
        self.controlador = controlador
        self.ventana_login = tk.Tk()
        self.ventana_login.title("Inicio de sesión")

        self.usuario = tk.StringVar()
        self.contrasena = tk.StringVar()

        tk.Label(self.ventana_login, text="Usuario").pack()
        tk.Entry(self.ventana_login, textvariable=self.usuario).pack()

        tk.Label(self.ventana_login, text="Contraseña").pack()
        tk.Entry(self.ventana_login, textvariable=self.contrasena, show='*').pack()

        tk.Button(self.ventana_login, text="Iniciar sesión", command=self.controlador.verificar_usuario).pack()

    def iniciar(self):
        self.ventana_login.mainloop()

    def cerrar(self):
        self.ventana_login.destroy()

    def ventana_pacientes(self):
        self.ventana_pacientes = tk.Tk()
        self.ventana_pacientes.title("Pacientes")

        self.id = tk.StringVar()
        self.nombre = tk.StringVar()
        self.apellido = tk.StringVar()
        self.edad = tk.StringVar()

        tk.Label(self.ventana_pacientes, text="ID").pack()
        tk.Entry(self.ventana_pacientes, textvariable=self.id).pack()

        tk.Label(self.ventana_pacientes, text="Nombre").pack()
        tk.Entry(self.ventana_pacientes, textvariable=self.nombre).pack()

        tk.Label(self.ventana_pacientes, text="Apellido").pack()
        tk.Entry(self.ventana_pacientes, textvariable=self.apellido).pack()

        tk.Label(self.ventana_pacientes, text="Edad").pack()
        tk.Entry(self.ventana_pacientes, textvariable=self.edad).pack()

        tk.Button(self.ventana_pacientes, text="Agregar paciente", command=self.controlador.agregar_paciente).pack()
        tk.Button(self.ventana_pacientes, text="Eliminar paciente", command=lambda: self.controlador.eliminar_paciente(self.id.get())).pack()

        self.busqueda = tk.StringVar()
        tk.Label(self.ventana_pacientes, text="Buscar").pack()
        tk.Entry(self.ventana_pacientes, textvariable=self.busqueda).pack()
        tk.Button(self.ventana_pacientes, text="Buscar paciente", command=self.controlador.buscar_pacientes).pack()

        self.ventana_pacientes.mainloop()

class Controlador:
    def __init__(self):
        self.modelo = Modelo()
        self.vista = Vista(self)

    def verificar_usuario(self):
        if self.modelo.verificar_usuario(self.vista.usuario.get(), self.vista.contrasena.get()):
            messagebox.showinfo("", "Inicio de sesión exitoso")
            self.vista.cerrar()
            self.vista.ventana_pacientes()
        else:
            messagebox.showinfo("", "Usuario o contraseña incorrectos")

    def agregar_paciente(self):
        if self.modelo.agregar_paciente(self.vista.id.get(), self.vista.nombre.get(), self.vista.apellido.get(), self.vista.edad.get()):
            messagebox.showinfo("", "Paciente agregado exitosamente")
        else:
            messagebox.showinfo("", "La identificación ya existe")

    def eliminar_paciente(self, id):
        if self.modelo.eliminar_paciente(id):
            messagebox.showinfo("", "Paciente eliminado exitosamente")
        else:
            messagebox.showinfo("", "La identificación no existe")

    def buscar_pacientes(self):
        resultados = self.modelo.buscar_pacientes(self.vista.busqueda.get())
        if resultados:
            for id, datos in list(resultados.items()):  
                if id not in self.modelo.pacientes:  
                    continue
                messagebox.showinfo("", f"ID: {id}, Nombre: {datos['nombre']}, Apellido: {datos['apellido']}, Edad: {datos['edad']}")
                if messagebox.askyesno("Eliminar", "¿Deseas eliminar este paciente?"):
                    self.eliminar_paciente(id)
        else:
            messagebox.showinfo("", "No se encontraron pacientes")

app = Controlador()
app.vista.iniciar()
